In [1]:
from BadWordDetectionByRegularExpression import return_bad_words_index
from JamoSplit import jamo_split

# model load
from gensim.models import FastText
from keras.models import load_model
from sklearn.externals import joblib

import numpy as np

embedding_model = FastText.load("./gensim_festtext.model")
cnn_model = load_model("./cnn_model")
rf_model = joblib.load("./rf_model")

Using TensorFlow backend.
/Users/daier/anaconda3/envs/testEnv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
W1018 17:35:39.563318 4502631872 deprecation_wrapper.py:119] From /Users/daier/anaconda3/envs/testEnv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1018 17:35:39.586373 4502631872 deprecation_wrapper.py:119] From /Users/daier/anaconda3/envs/testEnv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.unifor

# ==== 데이터를 input에 넣기 위한 전처리 ====

In [2]:
text="""
오늘은 2018년이다. 근데 저 18년이 나에게 욕을했다. 엠창 개새끼라고 이것들이 다 욕으로 처리될까요 완전 개새끼구만 오늘 아침에는 18년이 기분이 좋았다 올해 18년 계획은 지금 존나 졸리다 어떡하지 프로그램 미운우리새끼는 욕이 아니고 저 새끼는 욕이겠지 아마 삼시세끼도 똑같겠지? 또 뭐가 있을까요...... ㅋㅋㅋㅋㅋㅋㅋㅋ십알은 어떨까요
"""

In [3]:
trigram_list = return_bad_words_index(text, mode=1) # 욕설의 형태를 띄는 곳에가서 좌우단어 포함하여 trigram으로 반환

In [4]:
def chunks(l, n, trigram_list):
        '''
        vectroize 할 때 필요한 리스트를 청크별로 나누는 함수
        input : list, n(청크 단위)
        output : (n개씩 묶어진 list, word_index(단어위치))
        '''
        for i in range(0, len(l), n):
            yield (l[i:i + n], trigram_list[i//n][-1])

In [5]:
trigram_list

[('저', '새끼는', '욕이겠지', 32),
 ('\n오늘은', '2018년이다.', '근데', 1),
 ('아마', '삼시세끼도', '똑같겠지?', 35),
 ('저', '18년이', '나에게', 4),
 ('욕을했다.', '엠창', '개새끼라고', 7),
 ('엠창', '개새끼라고', '이것들이', 8),
 ('완전', '개새끼구만', '오늘', 14),
 ('아침에는', '18년이', '기분이', 17),
 ('올해', '18년', '계획은', 21),
 ('지금', '존나', '졸리다', 24),
 ('프로그램', '미운우리새끼는', '욕이', 28)]

In [6]:
# vectorize : trigram을 150차원의 벡터 + word index형태의 리스트로 만들어주는 과정 
trigram_vector = np.array([np.array(embedding_model[jamo_split(word)]) for trigram in trigram_list for word in trigram[:-1]])
trigram_vector = np.array(list(chunks(trigram_vector, 3, trigram_list))) # 50차원의 3개의 vector가 1개의 trigram에 들어가기위해 나눠주는 과정
trigram_vector = np.array([np.append(_[0].flatten(), _[1]) for _ in trigram_vector]) # 3 x 50 을 150차원으로 flatten + word index = 151dim

/Users/daier/anaconda3/envs/testEnv/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [7]:
trigram_vector.shape

(11, 151)

In [8]:
word_index = np.int8(trigram_vector[:, -1]) # word_index 단어위치를 뽑아내기
trigram_vector = np.delete(trigram_vector, -1, axis=1) # word_index 지우기

# ==== 결과 확인 ====
# ==== CNN ====

In [16]:
trigram_vector = trigram_vector.reshape(trigram_vector.shape[0], trigram_vector.shape[1], 1) # keras input 맞춰주기

In [17]:
trigram_vector.shape

(11, 150, 1)

In [18]:
# cnn
print("단어위치\n", word_index)
print("예측 확률 값\n", cnn_model.predict(trigram_vector))
result = cnn_model.predict(trigram_vector) > 0.65 # 0.65보다 높으면 욕설
result = result.reshape(-1).tolist()
print("Class와 단어 위치\n", list(zip(result, word_index.tolist())))

단어위치
 [32  1 35  4  7  8 14 17 21 24 28]
예측 확률 값
 [[0.9957671 ]
 [0.62791127]
 [0.70563596]
 [0.98375434]
 [0.99994034]
 [0.9999865 ]
 [0.9995563 ]
 [0.9997824 ]
 [0.00102557]
 [0.9999993 ]
 [0.9139524 ]]
Class와 단어 위치
 [(True, 32), (False, 1), (True, 35), (True, 4), (True, 7), (True, 8), (True, 14), (True, 17), (False, 21), (True, 24), (True, 28)]


In [19]:
# 결과 확인
print("비속어\n", np.array(trigram_list)[np.array(result)])
print("비속어 아닌것들\n", np.array(trigram_list)[np.array(result) == False])

비속어
 [['저' '새끼는' '욕이겠지' '32']
 ['아마' '삼시세끼도' '똑같겠지?' '35']
 ['저' '18년이' '나에게' '4']
 ['욕을했다.' '엠창' '개새끼라고' '7']
 ['엠창' '개새끼라고' '이것들이' '8']
 ['완전' '개새끼구만' '오늘' '14']
 ['아침에는' '18년이' '기분이' '17']
 ['지금' '존나' '졸리다' '24']
 ['프로그램' '미운우리새끼는' '욕이' '28']]
비속어 아닌것들
 [['\n오늘은' '2018년이다.' '근데' '1']
 ['올해' '18년' '계획은' '21']]


# ==== RandomForest ====

In [20]:
trigram_vector = trigram_vector.reshape(trigram_vector.shape[0], trigram_vector.shape[1]) # random forest input 맞춰주기

In [21]:
# randomforest
print("단어위치\n", word_index)
print("예측 확률 값\n", rf_model.predict_proba(trigram_vector))
result = rf_model.predict_proba(trigram_vector)[:, 1] > 0.65 # 0.65보다 높으면 욕설
result = result.tolist()
# result = [_==1 for _ in result] # Boolean list로 만들기
print("Class와 단어 위치\n", list(zip(result, word_index.tolist())))

단어위치
 [32  1 35  4  7  8 14 17 21 24 28]
예측 확률 값
 [[0.3375 0.6625]
 [0.275  0.725 ]
 [0.35   0.65  ]
 [0.3125 0.6875]
 [0.125  0.875 ]
 [0.1125 0.8875]
 [0.2    0.8   ]
 [0.1875 0.8125]
 [0.6125 0.3875]
 [0.0875 0.9125]
 [0.325  0.675 ]]
Class와 단어 위치
 [(True, 32), (True, 1), (False, 35), (True, 4), (True, 7), (True, 8), (True, 14), (True, 17), (False, 21), (True, 24), (True, 28)]


In [22]:
# 결과 확인
print("비속어\n", np.array(trigram_list)[np.array(result)])
print("비속어 아닌것들\n", np.array(trigram_list)[np.array(result) == False])

비속어
 [['저' '새끼는' '욕이겠지' '32']
 ['\n오늘은' '2018년이다.' '근데' '1']
 ['저' '18년이' '나에게' '4']
 ['욕을했다.' '엠창' '개새끼라고' '7']
 ['엠창' '개새끼라고' '이것들이' '8']
 ['완전' '개새끼구만' '오늘' '14']
 ['아침에는' '18년이' '기분이' '17']
 ['지금' '존나' '졸리다' '24']
 ['프로그램' '미운우리새끼는' '욕이' '28']]
비속어 아닌것들
 [['아마' '삼시세끼도' '똑같겠지?' '35']
 ['올해' '18년' '계획은' '21']]
